In [1]:
import pandas as pd
import numpy as np

In [2]:
#Загрузчик с диска гугл
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
#!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#!pip install xlrd>=1.2.0

In [4]:
df = pd.read_excel('../content/drive/MyDrive/Colab Notebooks/NLP/7/отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 8.2 MB 51.5 MB/s 


In [7]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=ab85c24813dfae4676bcd3340bf1efb65ed4155b22a9cfb1e31b577b6455bd5e
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [8]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [10]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [12]:
train_corpus = " ".join(df['Content'])
train_corpus = train_corpus.lower()

In [13]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [14]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [15]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [16]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [17]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [18]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [37]:
df["Rating"] = df["Rating"] - 1

In [38]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)
df_test, df_val = train_test_split(df_test, test_size=0.2)

In [39]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [40]:
x_train.shape

(16527, 40)

In [23]:
x_test.shape

(3305, 40)

In [24]:
x_val.shape

(827, 40)

In [25]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   7,
       163], dtype=int32)

Keras model

In [27]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  

In [87]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import Constant

In [41]:
num_classes = 5
y_train = to_categorical(df_train["Rating"], num_classes)
y_val = to_categorical(df_val["Rating"], num_classes)

In [89]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [90]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [91]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 6s 189ms/step - loss: 1.1776 - accuracy: 0.6732 - val_loss: 0.9553 - val_accuracy: 0.7108
Epoch 2/20
30/30 [==============================] - 5s 183ms/step - loss: 0.8538 - accuracy: 0.7055 - val_loss: 0.7445 - val_accuracy: 0.7108


In [92]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 34ms/step - loss: 0.7572 - accuracy: 0.6965


Test score: 0.7571641206741333
Test accuracy: 0.6964933276176453


In [49]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 0s 52ms/step


Предобученная модель

In [80]:
!pip install Navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
path = '../content/drive/MyDrive/Colab Notebooks/NLP/7/navec_news_v1_1B_250K_300d_100q.tar'
emb = Navec.load(path)

In [85]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [88]:
embedding_layer = Embedding(
    max_words,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [93]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [95]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [96]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 9s 283ms/step - loss: 0.9419 - accuracy: 0.7120 - val_loss: 0.7949 - val_accuracy: 0.7459
Epoch 2/20
30/30 [==============================] - 8s 280ms/step - loss: 0.7693 - accuracy: 0.7442 - val_loss: 0.7338 - val_accuracy: 0.7556


In [97]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 76ms/step - loss: 0.7375 - accuracy: 0.7485


Test score: 0.737492561340332
Test accuracy: 0.7484884858131409


С предобученной моделью результат лучше, при том по минимальной структуре сетки